In [1]:
import numpy as np
import matplotlib as plt
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
np.random.seed(1)

In [21]:
def zero_pad(X,pad):
    X_pad = np.pad(X, ((0,0), (pad, pad), (pad, pad),(0,0)),'constant', constant_values = 0)
    return X_pad

In [22]:
X=np.random.randn(4,3,3,2)
X_pad=zero_pad(X,2)
print(X_pad)

[[[[ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.42349435  0.07734007]
   [-0.34385368  0.04359686]
   [-0.62000084  0.69803203]
   [ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]
   [-0.44712856  1.2245077 ]
   [ 0.40349164  0.59357852]
   [-1.09491185  0.16938243]
   [ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]
   [ 0.74055645 -0.9537006 ]
   [-0.26621851  0.03261455]
   [-1.37311732  0.31515939]
   [ 0.          0.        ]
   [ 0

In [23]:
def conv_single_step(a_slice_prev,W,b):
    Z=a_slice_prev*W+b
    a=np.sum(Z)
    return a

def conv_forward(A_prev,W,b,hparameters):
    (m,n_Hprev,n_Wprev,n_C_p)=A_prev.shape
    (f,f,n_C_prec,n_c)=W.shape
    stride=hparameters["stride"]
    pad=hparameters["pad"]
    
    

In [25]:
def conv_forward(A_prev,W,b,hparameters):
    (m,n_Hprev,n_Wprev,n_C_p)=A_prev.shape
    (f,f,n_C_prec,n_c)=W.shape
    stride=hparameters["stride"]
    pad=hparameters["pad"]
    A_prev=zero_pad(A_prev,pad)
    n_H=int((n_Hprev+2*pad-f)/stride)+1
    n_W=int((n_Wprev+2*pad-f)/stride)+1
    Z=np.zeros(m,n_H,n_W,n_c)
    for batch in range(m):
        a_prev=a_prev[batch]
        for h in range(n_H):
            for w in range(n_W):
                for ch in range(n_c):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    a_slice_prev=a_prev[vert_start:vert_end,horiz_start:horiz_end,:]
                    Z[batch,h,w,ch]=np.sum(np.multiply(a_slice_prev*W[:,:,:,ch])+b[:,:,:,ch])
    cache=(A_prev,W,b,hparameters)
    return Z,cache
            
    

In [26]:
def pool_forward(A_prev,hparameters,mode='max'):
    (m,n_Hprev,n_Wprev,n_C_p)=A_prev.shape
    stride=hparameters["stride"]
    f=hparameters["f"]
    n_H=int((n_Hprev-f)/stride)+1
    n_W=int((n_Wprev-f)/stride)+1
    Z=np.zeros(m,n_H,n_W,n_c)
    for batch in range(m):
        a_prev=a_prev[batch]
        for h in range(n_H):
            for w in range(n_W):
                for ch in range(n_c):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    a_slice_prev=a_prev[vert_start:vert_end,horiz_start:horiz_end,:]
                    if(mode=="max"):
                        Z[batch,h,w,ch]=np.max(a_slice_prev)
                    else:
                        Z[batch,h,w,ch]=np.mean(a_slice_prev)
    cache=(A_prev,hparameters)
    return Z,cache
    